## 爬取腾讯的疫情数据

In [1]:
import requests
import json
url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
res = requests.get(url)
d = json.loads(res.text)
print(d.keys())

dict_keys(['ret', 'data'])


In [2]:
data_all = json.loads(d['data'])
print(data_all.keys())

dict_keys(['lastUpdateTime', 'chinaTotal', 'chinaAdd', 'isShowAdd', 'showAddSwitch', 'areaTree'])


In [3]:
print(len(data_all['areaTree']))
print(data_all['areaTree'][0].keys())

1
dict_keys(['name', 'today', 'total', 'children'])


In [4]:
print(data_all['areaTree'][0]['name'])
print(data_all['areaTree'][0]['today'])
print(data_all['areaTree'][0]['total'])

中国
{'confirm': 16, 'isUpdated': True}
{'nowConfirm': 535, 'confirm': 92492, 'suspect': 0, 'dead': 4749, 'deadRate': '5.13', 'showRate': False, 'heal': 87208, 'healRate': '94.29', 'showHeal': True}


In [5]:
print(len(data_all['areaTree'][0]['children']))

for i in data_all['areaTree'][0]['children']:
    print(i['name'])

34
香港
上海
台湾
四川
陕西
广东
福建
内蒙古
天津
江苏
新疆
湖北
辽宁
山西
浙江
云南
河南
北京
河北
甘肃
山东
安徽
湖南
广西
重庆
西藏
吉林
宁夏
江西
贵州
海南
青海
澳门
黑龙江


In [6]:
import pymysql
import time
import json
import traceback

In [7]:
def get_tencent_data():
    url1 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    url2 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
    }
    r1 = requests.get(url1, headers)
    r2 = requests.get(url2, headers)

    res1 = json.loads(r1.text)
    res2 = json.loads(r2.text)

    data_all1 = json.loads(res1["data"])
    data_all2 = json.loads(res2["data"])

    history = {}
    for i in data_all2["chinaDayList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds, "%Y.%m.%d")  # 匹配时间
        ds = time.strftime("%Y-%m-%d", tup)  # 改变时间格式
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        history[ds] = {"confirm": confirm, "suspect": suspect, "heal": heal, "dead": dead}
    for i in data_all2["chinaDayAddList"]:
        ds = "2020." + i["date"]
        tup = time.strptime(ds, "%Y.%m.%d")  # 匹配时间
        ds = time.strftime("%Y-%m-%d", tup)  # 改变时间格式
        confirm = i["confirm"]
        suspect = i["suspect"]
        heal = i["heal"]
        dead = i["dead"]
        history[ds].update({"confirm_add": confirm, "suspect_add": suspect, "heal_add": heal, "dead_add": dead})

    details = []
    update_time = data_all1["lastUpdateTime"]
    data_country = data_all1["areaTree"]
    data_province = data_country[0]["children"]
    for pro_infos in data_province:
        province = pro_infos["name"]
        for city_infos in pro_infos["children"]:
            city = city_infos["name"]
            confirm = city_infos["total"]["confirm"]
            confirm_add = city_infos["today"]["confirm"]
            heal = city_infos["total"]["heal"]
            dead = city_infos["total"]["dead"]
            details.append([update_time, province, city, confirm, confirm_add, heal, dead])
    return history, details

## 存储数据

In [9]:
def get_conn():
    # 建立连接
    conn = pymysql.connect(host='localhost',
                          user='root',
                          password='123456',
                          db='cov')
    # 创建游标
    cursor = conn.cursor()
    return conn, cursor

def close_conn(conn, cursor):
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [10]:
def update_details():
    """更新details表"""
    cursor = None
    conn = None
    try:
        li = get_tencent_data()[1]
        conn, cursor = get_conn()
        sql = 'insert into details(update_time,province,city,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s,%s)'
        sql_query = 'select %s=(select update_time from details order by id desc limit 1)' # 对比当前最大时间戳
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:
            print(f'{time.asctime()}开始更新最新数据')
            for item in li:
                cursor.execute(sql,item)
            conn.commit()
            print(f'{time.asctime()}更新最新数据完毕')
        else:
            print(f'{time.asctime()}已是最新数据')
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [11]:
def insert_history():
    """插入history表"""
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]
        print(f'{time.asctime()}开始插入历史数据')
        conn, cursor = get_conn()
        sql = 'insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        for k,v in dic.items():
            cursor.execute(sql,[k,v.get('confirm'),v.get('confirm_add'),v.get('suspect'),
                                v.get('suspect_add'),v.get('heal'),v.get('heal_add'),
                                v.get('dead'),v.get('dead_add')])
        conn.commit()
        print(f'{time.asctime()}插入历史数据完毕')
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [12]:
def update_history():
    """插入history表"""
    cursor = None
    conn = None
    try:
        dic = get_tencent_data()[0]
        print(f'{time.asctime()}开始更新历史数据')
        conn, cursor = get_conn()
        sql = 'insert into history values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        sql_query = 'select confirm from history where ds=%s'
        for k,v in dic.items():
            cursor.execute(sql,[k,v.get('confirm'),v.get('confirm_add'),v.get('suspect'),
                                v.get('suspect_add'),v.get('heal'),v.get('heal_add'),
                                v.get('dead'),v.get('dead_add')])
        conn.commit()
        print(f'{time.asctime()}历史数据更新完毕')
    except:
        traceback.print_exc()
    finally:
        close_conn(conn, cursor)

In [16]:
# 插入历史数据
insert_history()

Wed Nov 18 17:26:16 2020开始插入历史数据
Wed Nov 18 17:26:16 2020插入历史数据完毕


In [15]:
# 建立连接
conn = pymysql.connect(host='localhost',
                      user='root',
                      password='123456',
                      db='cov')
# 创建游标，默认是元组型
cursor = conn.cursor()
sql = 'select * from history limit 5'
cursor.execute(sql)
# 提交事务
res = cursor.fetchall()
print(res)
cursor.close()
conn.close()

((datetime.datetime(2020, 1, 13, 0, 0), 41, None, 0, None, 0, None, 1, None), (datetime.datetime(2020, 1, 14, 0, 0), 41, None, 0, None, 0, None, 1, None), (datetime.datetime(2020, 1, 15, 0, 0), 41, None, 0, None, 5, None, 2, None), (datetime.datetime(2020, 1, 16, 0, 0), 45, None, 0, None, 8, None, 2, None), (datetime.datetime(2020, 1, 17, 0, 0), 62, None, 0, None, 12, None, 2, None))


In [17]:
update_details()

Wed Nov 18 17:26:53 2020开始更新最新数据
Wed Nov 18 17:26:53 2020更新最新数据完毕
